# Dimensionality reduction 2

__Week 2 - 4 May 2022__

After we have practised using some dimensionality reduction algorithms in basic mode, we will now turn our attention to arguably the most important part of the process - preprocessing and hyperparameter optimisation. Hopefully this will show how these steps can be crucial in producing the best possible representation of data in lower dimensions. Here, we will make use of a new astronomical dataset with the same algorithms as before (`PCA`, `t-SNE`, `UMAP`). We will end by returning to galaxies in COSMOS2015 and improving on the results from the previous session.

---
### Data

Swift dataset - the dataset is light curves (intensity of light on a timeline) of gamma-ray bursts (GRB) coming from the far reaches of the universe. In addition to the raw data, there are several derived properties:

* `T90_start` & `T90_end` - start and end times of the bursts of light.
* `fluence`
* `hardness`

---
### Exercise

You have now practiced reducing a dataset to a set of basis vectors. The next step is to take an additional step of considering which data it is best to use in the first place and how to prepare it.

* Apply your favourite dimensionality reduction algorithm to: (1) properties, (2) raw observed data, (3) preprocessed observed data in the Swift dataset.

* Tweak hyperparameters of the algorithm - can you find an optimal combination?

* P.S. Why use only one of the algorithms at a time? Think about using one of these for preprocessing and the other for dimensionality reduction (eg., `PCA` + `t-SNE`).


---
* Authors:  Vadim Rusakov, Charles Steinhardt, Jackson Mann
* Email:  vadim.rusakov@nbi.ku.dk
* Date:   27th of April 2022

In [ ]:
!conda install -c conda-forge umap-learn

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - umap-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py37_0         148 KB  conda-forge
    umap-learn-0.4.6           |   py37hc8dfbb8_0         110 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         259 KB

The following NEW packages will be INSTALLED:

  umap-learn         conda-forge/osx-64::umap-learn-0.4.6-py37hc8dfbb8_0

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi                                         p

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import umap
from matplotlib.colors import LogNorm
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, KernelPCA

Load the raw data (light curves), as well as the properties (derived form raw data). 

Make sure to get rid of the `NaN` entries.

In [ ]:
# loading data (light curves)
data = pd.read_pickle('datasets/swift_gamma_ray_bursts.zip')

# loading derived properties
props_full = pd.read_csv('datasets/SwiftProperties.csv')

# match & merge properties and raw data tables
df_merged = pd.merge(props_full, data, left_on='GRBname', right_index=True, how='inner')
mask = ~np.isnan(df_merged.iloc[:, 1:]).any(axis=1) # mask of NaNs
df_clean = df_merged.loc[mask] # clean of NaNs
X_data = df_merged.values[mask, 7:] # raw data
X_props = df_merged.values[mask, :7] # derived properties

print(f"the shape of the raw data set (nsamples, nfeatures): {X_data.shape}")
print(f"the shape of the properties data set (nsamples, nfeatures): {X_props.shape}")

## Dataset 1. Gamma-ray bursts

1. Run your favourite algorithm (`t-SNE` or `UMAP`) on the bursts properties (`X_props`). Does it manage to split the samples into two types: eg., short and long bursts? $^1$

2. Can you explain the clumpiness of the reduction? Based on this, do derived prperties give a meaningful reduction? In not, can you argue why?


$^1$ Hint: short- or long-type of bursts separation is not obvious. Calculate the duration of bursts using the properties table and by subtracting `T90_start` from `T90_end`. Colour your scatter plots in reduced dimensions by duration. Try other properties as well.

In [ ]:
# running t-SNE
tsne = TSNE(perplexity=15, n_iter=1500, init='random', verbose=2, 
            learning_rate='auto', method='barnes_hut', random_state=42)
y = tsne.fit_transform(X_props[:, 1:])

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 5), dpi=100)
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2')

duration = np.log10(df_clean['T90_end'] - df_clean['T90_start'])

sc = ax.scatter(y[:, 0], y[:, 1], s=2.0, norm=LogNorm(), 
                c=duration, cmap='jet')
cbar = plt.colorbar(sc)
cbar.ax.set_ylabel('Duration', rotation=270, labelpad=10)
plt.show()

Reapeat with `UMAP`:

In [ ]:
map = umap.UMAP(n_components=2, n_neighbors=50, random_state=42)
y = map.fit_transform(X_props[:, 1:])

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 5), dpi=100)
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2')

sc = ax.scatter(y[:, 0], y[:, 1], s=1.0, norm=LogNorm(), 
                c=duration,  cmap='jet')
cbar = plt.colorbar(sc)
cbar.ax.set_ylabel('Duration', rotation=270, labelpad=10)
plt.show()

3. Now train the algorithm with the raw data `X_data`. Are you able to make cleaner classifications using the projections in the space learnt by the algorithms? If not, do you have some ideas for how to improve this?

In [ ]:
map = umap.UMAP(n_components=2, n_neighbors=50, random_state=42)
y = map.fit_transform(X_data)

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 5), dpi=100)
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2')

duration = np.log10(df_clean['T90_end'] - df_clean['T90_start'])

sc = ax.scatter(y[:, 0], y[:, 1], s=2.0, norm=LogNorm(), 
                c=duration, cmap='jet')
cbar = plt.colorbar(sc)
cbar.ax.set_ylabel('Duration', rotation=270, labelpad=10)
plt.show()

4. Suggestion: try converting the time-series `X_data` to frequency space by taking a Fourier transform. Repeat the exercise. Does this help to create a cleaner representation in the reduced space? What physical meaning does the separation imply? (for eg., in terms of the burst duration property).

In [ ]:
# taking Fast Fourier Transform
X_data_fft = abs(np.fft.rfft(X_data)) # take modulus to convert to float from complex data type
mask_fft = ~np.isnan(X_data_fft[:, 1:]).any(axis=1) # NaNs
X_data_mod = X_data_fft[mask_fft] # mask out NaNs

In [ ]:
# running t-SNE
tsne = TSNE(perplexity=15, n_iter=1500, init='random', verbose=2, 
            learning_rate='auto', method='barnes_hut', random_state=42)
y = tsne.fit_transform(X_data_mod)

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 5), dpi=100)
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2')
duration = np.log10(df_clean['T90_end'] - df_clean['T90_start'])
duration = duration.iloc[mask_fft]

sc = ax.scatter(y[:, 0], y[:, 1], s=2.0, norm=LogNorm(), 
                c=duration, cmap='jet')
cbar = plt.colorbar(sc)
cbar.ax.set_ylabel('Duration', rotation=270, labelpad=10)
plt.show()

5. Try experimenting: the main parameter in `t-SNE` is `perplexity` - try using the values of `5, 10, 15, 20, 50`. The analogous parameter for `UMAP` is `n_neighbours`. Can you find a number that makes an optimal separation of bursts? How would this number have to change depending on the size of the dataset, eg. if you have **10x**, **100x** more samples?

6. Try changing the different options `method='exact'` $\left[ \mathcal{O}(n^2) \right]$ and `method='barnes_hut'` $\left[ \mathcal{O}(n \log{n}) \right]$. The complexity of the operations with the two is different: `barnes_hut` provides an approximate map, but with less time spent.

7. Try different value of the number of iterations (`n_iter`). Can you reach the point where the algorithm converges (be gentle to your computer)? Remember that you need to have at least `n_iter = 250`, as the algorithm requires those for calibration. If it takes too long to converge try taking a smaller sub-sample.

### Back to COSMOS2015 (galaxies)

Now, repeat the experiment from the previous session with the galaxies dataset. This time think about preprocessing and hyperparameter optimisation and repeat the exercise. At the end try to classify galaxies as either `alive` or `dead`.

1. Recall that the linear `PCA` before was not capable of linearly separating two types of galaxies at all. In fact, the data you were given were not entirely ready. You need to preprocess them first. For this, take all fluxes and normalize them by one of the other fluxes. $^{1,2}$ Repeat the exercise. Do you get better separation?

$^1$ Hint: it turns out the star forming and dead galaxies are most different in the bluer bands (like `u_flux`).\
$^2$ Hint: plot distributions of features before preprocessing and compare with the same after.

In [ ]:
# turn off warnings, no one needs them...
pd.options.mode.chained_assignment = None  # default='warn'

file = "datasets/cosmos2015.csv"
df = pd.read_csv(file, index_col=False)

# select a random sub-sample of the dataset
n = 10000
idxs = np.arange(df.shape[0])
idxs_rand = np.random.choice(idxs, size=n)
df_cut = df.iloc[idxs_rand] # dataframe
X = df.iloc[idxs_rand].values # array

flux_cols = list(df.columns[4:]) # flux column names
flux_idxs = np.argwhere(np.isin(df.columns, flux_cols)).flatten() # flux column indices

In [ ]:
fig, ax = plt.subplots(1)
for i, col in enumerate(flux_cols):
    vec = X[:, i]
    vec = vec[(vec > (-10)) & (vec < 60)]
    ax.hist(vec, bins=100, label=col, density=True, histtype='step')
ax.set_ylim(0, 0.1)
plt.legend()
plt.show()

In [ ]:
# normalize fluxes 
norm_flux = df_cut.loc[:, 'u_flux'].values
X_trans = np.copy(X)
for col in flux_idxs:
    X_trans[:, col] = X_trans[:, col] / norm_flux

The result of the normalization is below. The distributions are a little more closely spaced and span the similar numerical ranges.

In [ ]:
fig, ax = plt.subplots(1)
for i, col in enumerate(flux_cols):
    vec = X_trans[:, i]
    vec = vec[(vec > (-10)) & (vec < 30)]
    ax.hist(vec, bins=100, label=col, density=True, histtype='step')
ax.set_ylim(0, 0.1)
ax.set_xlim(-10, 60)
plt.legend()
plt.show()

In [ ]:
pca = PCA(n_components=2, svd_solver='full')
y_pcs = pca.fit_transform(X_trans[:, flux_idxs]) # train only on fluxes (raw observed data)

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 5), dpi=100)
ax.set_xlim(np.percentile(y_pcs[:,0], 99), np.percentile(y_pcs[:,0], 1))
ax.set_ylim(np.percentile(y_pcs[:,1], 99), np.percentile(y_pcs[:,1], 1))
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2')
is_sf = np.isin(df_cut.loc[:, 'is_star_forming'], 1)

ax.scatter(y_pcs[is_sf, 0], y_pcs[is_sf, 1], s=0.02, c='b', norm=LogNorm())
ax.scatter(y_pcs[~is_sf, 0], y_pcs[~is_sf, 1], s=0.02, c='r', norm=LogNorm())
ax.annotate("star forming", xy=(0.05, 0.9), xycoords="axes fraction", 
            color='b', fontsize=12)
ax.annotate("dead", xy=(0.05, 0.86), xycoords="axes fraction", 
            color='r', fontsize=12)
plt.show()

### t-SNE

In [ ]:
tsne = TSNE(n_components=2, init='random', method='barnes_hut',   
                     random_state=0, perplexity=10, n_iter=400, verbose=2)
y = tsne.fit_transform(X_trans[:, flux_idxs])

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 5), dpi=100)
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2')
is_sf = np.isin(df_cut.loc[:, 'is_star_forming'], 1)

ax.scatter(y[is_sf, 0], y[is_sf, 1], s=0.05, c='b', norm=LogNorm())
ax.scatter(y[~is_sf, 0], y[~is_sf, 1], s=0.05, c='r', norm=LogNorm())
ax.annotate("star forming", xy=(0.05, 0.9), xycoords="axes fraction", 
            color='b', fontsize=12)
ax.annotate("dead", xy=(0.05, 0.86), xycoords="axes fraction", 
            color='r', fontsize=12)
plt.show()

### UMAP

In [ ]:
map = umap.UMAP(n_components=2, random_state=42)
y = map.fit_transform(X_trans[:, flux_idxs])

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 5), dpi=100)
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2')
is_sf = np.isin(df_cut.loc[:, 'is_star_forming'], 1)

ax.scatter(y[is_sf, 0], y[is_sf, 1], s=0.05, c='b', norm=LogNorm())
ax.scatter(y[~is_sf, 0], y[~is_sf, 1], s=0.05, c='r', norm=LogNorm())
ax.annotate("star forming", xy=(0.05, 0.9), xycoords="axes fraction", 
            color='b', fontsize=12)
ax.annotate("dead", xy=(0.05, 0.86), xycoords="axes fraction", 
            color='r', fontsize=12)
plt.show()

2. Even with a high degree of separation, whether star forming or dead, there is a continuum of galaxies with varying masses and rates of star formation. Therefore, do you ever expect there to be discrete islands of galaxies when you reduce the dimensions? $^1$

$^1$ Hint: produce scatter plots in the reduced space and colour the points by different galaxy properties.

## PCA + UMAP

4. Try experimenting: can you combine `PCA` (or `kPCA`) and `t-SNE` as part of the preprocessing and training procedures? Do you get better results?

5. Apply the new manifold-learning techniques to the old COSMOS2015 dataset. Now can you tell galaxies alive from dead better? Colouring the points by galaxy properties, what can you learn about active or dead galaxies (eg., which masses or rates of star formation do they tend to have)?

Let us try to use a combination of PCA for preprocessing and UMAP for further reducing dimensions.

In [ ]:
# preprocessing
kpca = KernelPCA(n_components=8, kernel='poly')
y_pcs = kpca.fit_transform(X_trans[:, flux_idxs])

# reducing dimensions to 2 components
map = umap.UMAP(n_components=2, random_state=42)
y = map.fit_transform(y_pcs)

In [ ]:
fig, ax = plt.subplots(1, figsize=(6, 5), dpi=100)
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2')
is_sf = np.isin(df_cut.loc[:, 'is_star_forming'], 1)

ax.scatter(y[is_sf, 0], y[is_sf, 1], s=0.05, c='b', norm=LogNorm())
ax.scatter(y[~is_sf, 0], y[~is_sf, 1], s=0.05, c='r', norm=LogNorm())
ax.annotate("star forming", xy=(0.05, 0.9), xycoords="axes fraction", 
            color='b', fontsize=12)
ax.annotate("dead", xy=(0.05, 0.86), xycoords="axes fraction", 
            color='r', fontsize=12)
plt.show()